# BIO-SELECT - Marigliano
## Global pipeline

_TODO_ : insert global pipeline image here

## Imports

In [ ]:
from sklearn import neighbors, datasets
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
from sklearn import preprocessing

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

%matplotlib inline

# set float precision at 2 digits
np.set_printoptions(precision=2)

## Constants

In [ ]:
N_FEATURES_ALGORITHM = 1000

## Dataset loading
_TODO_: 
* this notebook must only load one dataset
* retrieve dataset to load from cmd arguments or from env variable

In [ ]:
from datasets.EGEOD22619.EGEOD22619Dataset import EGEOD22619Dataset
from datasets.MILE.MileDataset import MileDataset
from datasets.Golub99.GolubDataset import GolubDataset

from datasets.DatasetEncoder import DatasetEncoder
from datasets.DatasetSplitter import DatasetSplitter

#ds = MileDataset()
#ds = EGEOD22619Dataset()
ds = GolubDataset()

## Dataset transformation
The dataset needs some transformations such as encoding the outputs as float (necessary for scikit learn), normalization, ...

_TODO_:
* dataset splitting (train, test[, validation])
* encode outputs
* normalization
* classes merging
    * due to the low class balancing we might want to regroup them. Example Healthy vs Non-Healthy (choose the most represented class ?)

In [ ]:
# encode Dataset string classes into numbers
ds_encoder = DatasetEncoder(ds)
ds = ds_encoder.encode()
ds = DatasetSplitter(ds, test_size=0.4)

X = ds.get_X()
y = ds.get_y()
print(len(y))

X_train = ds.get_X_train()
y_train = ds.get_y_train()
X_test = ds.get_X_test()
y_test = ds.get_y_test()

print(len(X_train[0]))

## Algorithms
Run the chosen algorithms and save them and their output subset of features using cPickle into files. They can be used later to display some graphs and to be analyzed

_TODO_: Write a subsection for each algorithm :
* OneVsRest or OneVsOne ?
    * only for those who needs it
* Grid search + CV
    * maybe not for all algorithms such as SVM RFE which takes a lot of time
    * not for algorthms which does not have parameters to tune (ReliefF, Fisher Score,...)
* print classification report (accuracy, recall, precision, ...)
    * issue: not all algortihms are able to do this
* normalize score using minmax normalization (0-1)
* show score per features (50 to 100 first ones)
* save algorithm in a file

Algorithms:
* ExtraTrees
* Random Forest
* SVM
* SVM RFE
* ANN
* ReliefF
* Fisher Score
* "Best features subset ~ SVM"
* SVM Backward ?
* CFS - Correlation-based Feature Selection
* Mutual Information Classifier
* One genetic based algorithm

In [ ]:
from algorithms.ExtraTreesAlgorithm import ExtraTreesAlgorithm

eta = ExtraTreesAlgorithm(ds)

feats_by_score = eta.get_best_features_by_score(n=N_FEATURES_ALGORITHM)
print(feats_by_score)

## Features subsets merging
Each algorithm has done its work and provided a subset of features as:
* a ranked score list
* a ranked list (no score)
* a list (no ranking, no score)

This part uses some techniques to combine/merge theses lists into a better one

_TODO_: 
* implement merge techniques
    * votation
    * weighted votation
    * union of intersection
    * ...
* Visualize the lists
    * Venn diagram ?

## Evaluation of the merged subset
Once we have a merged list containing the best features, we would like to evaluate it with several classifiers

_TODO_: use a separate test set ? -> split again train/test set -> no changes in the Dataset class